In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

%run k2400.ipynb
%run srs.ipynb
%run fileio.ipynb


# IV is a data structure containing all data collected in a given execution of lockin_iv.
class IV:
    def __init__(self, dc_voltage, delay):
        self.dc_voltage = dc_voltage # array of dc voltages
        self.dc_current = np.zeros(len(dc_voltage)) # array of dc current readings from keithley
        self.ac_voltagexx = np.zeros(len(dc_voltage)) # array of ac voltage readings from lockin
        self.ac_voltagexy = np.zeros(len(dc_voltage)) # array of ac voltage readings from lockin
        self.ac_current = np.zeros(len(dc_voltage)) # array of ac voltage readings from lockin
        self.phase = np.zeros(len(dc_voltage)) # array of phase readings from lockin
        self.delay = delay # acquisition delay 
        

# lockin_iv is a function that sequentially sets a keithley to a specified array of voltages. at each
# voltage, the dc current on the keithley and the ac voltage and phase read by the lockin are recorded.
# at the end of execution, the data will be saved to an IV object (defined above). this IV object will
# also be written to a file for later retrieval.

# input parameters:
#    k2400 is a keithley object, assumed initialized
#    srs is a srs lockin object, assumed initialized
#    dc_voltage is an array of dc voltages for the keithley
#    delay is the time to delay before acquiring data at each dc voltage
def lockin_iv_keithley(k2400,srsxx,srsxy,srscurrent,dc_voltage,delay,step):
    %matplotlib inline
    
    # create new IV object to store data in
    iv = IV(dc_voltage,delay)
    iv2 = IV(dc_voltage,delay)
    
    # make a filename to save the data, for later use
    full_filename = makefilename(None, 1, step)
    full_filename2 = makefilename(None, 2, step)

    # define a figure with two panels to show the data being acquired
    fig = plt.figure(figsize=(8, 3)) 
    
    fig.suptitle(full_filename)
    
    ax1=plt.subplot(2,2,1)
    ax1.set_xlabel('dc gate voltage (V)')
    ax1.set_ylabel('ac voltage xx (A)')
    plt.plot(iv.dc_voltage,iv.ac_voltagexx)
    
    ax2=plt.subplot(2,2,2)
    ax1.set_xlabel('dc gate voltage (V)')
    ax2.set_ylabel('dc current (A)')
    plt.plot(iv.dc_voltage,iv.dc_current)
    
    ax3=plt.subplot(2,2,3)
    ax1.set_xlabel('dc gate voltage (V)')
    ax1.set_ylabel('ac voltage xx (A)')
    plt.plot(iv.dc_voltage,iv.ac_voltagexx)
    
    ax4=plt.subplot(2,2,4)
    ax1.set_xlabel('dc gate voltage (V)')
    ax4.set_ylabel('dc current (A)')
    plt.plot(iv.dc_voltage,iv.dc_current)
    
    plt.tight_layout()
    

    
    try:
        # the for loop below is the main acquisition loop. the reason the for loop lives in a try-except
        # is that if the user cancels while the sweep is running, which throws an exception, the program 
        # catches it and continues to execute below the except statement. this lets the program shut down
        # nicely and save what data has already been collected.
        
        # main loop: iterate over dc voltages
        for i in range(len(iv.dc_voltage)):
            # set voltage
            sweep2400(k2400,iv.dc_voltage[i])
            
            # wait acquisition delay 
            time.sleep(iv.delay)
            
            # read data and store it
            iv.dc_current[i]=iv2.dc_current[i]=read2400(k2400)[1]
            iv.ac_voltagexx[i],iv.phase[i],iv2.ac_voltagexx[i]=readsrs(srsxx)
            iv2.phase[i] = iv.phase[i]
            
            # update plots
            ax1.lines[0].set_xdata(iv.dc_voltage[0:(i+1)])
            ax1.lines[0].set_ydata(iv.ac_voltagexx[0:(i+1)])
            ax1.relim()
            ax1.autoscale()

            ax2.lines[0].set_xdata(iv.dc_voltage[0:(i+1)])
            ax2.lines[0].set_ydata(iv.dc_current[0:(i+1)])
            ax2.relim()
            ax2.autoscale()
            
            ax3.lines[0].set_xdata(iv.dc_voltage[0:(i+1)])
            ax3.lines[0].set_ydata(iv.ac_voltagexx[0:(i+1)])
            ax3.relim()
            ax3.autoscale()

            ax4.lines[0].set_xdata(iv.dc_voltage[0:(i+1)])
            ax4.lines[0].set_ydata(iv.dc_current[0:(i+1)])
            ax4.relim()
            ax4.autoscale()
            
            plt.tight_layout()

            display.clear_output(wait=True)
            display.display(plt.gcf())
    except: 
        ; # do nothing if there was an exception. just proceed to what follows the try-except statement
        
    display.clear_output(wait=True)
    
    # save data as a .pkl file for use with python
    savedata(full_filename,iv)
    savedata(full_filename2,iv2)
    
    # also save data as a text file for easy opening with other software
    np.savetxt(full_filename+'.txt',np.transpose(np.array([iv.dc_voltage,iv.dc_current,iv.ac_voltagexx,iv.phase])),newline='\r\n')
    np.savetxt(full_filename2+'.txt',np.transpose(np.array([iv2.dc_voltage,iv2.dc_current,iv2.ac_voltagexx,iv2.phase])),newline='\r\n')

    
    # zero the dc voltage
    sweep2400(k2400,0)
    
    return [iv,iv2]

